In [1]:
!pip install -q unsloth
!pip uninstall unsloth -y && pip install -q --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 140.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, json, torch
from datasets import Dataset
import numpy as np

print("=" * 60)
print("🔧 SİSTEM BİLGİSİ")
print("=" * 60)
print("PyTorch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())
if not torch.cuda.is_available():
    raise RuntimeError("❌ GPU gerekli!")
print("GPU:", torch.cuda.get_device_name(0))
print("VRAM(GB):", torch.cuda.get_device_properties(0).total_memory / 1e9)

🔧 SİSTEM BİLGİSİ
PyTorch: 2.9.1+cu128
CUDA: True
GPU: NVIDIA A100-SXM4-80GB
VRAM(GB): 85.167243264


In [3]:
# 📁 VERİ DOSYASI
JSONL_FILE = "/content/drive/MyDrive/mody_dataset_with_system.jsonl"

# 🤖 MODEL
MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct"
MAX_SEQ_LENGTH = 1536

# 💾 ÇIKTI
OUTPUT_DIR = "./mody-llama-son"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 🎯 EĞİTİM - ⚡ OPTİMİZE
NUM_EPOCHS = 3              # ✅ Aynı (senin tercihin)
LEARNING_RATE = 2.5e-5      # ✅ Aynı

# ⚡ BATCH - 80GB için optimize (gradient offload önleme)
BATCH_SIZE = 8            # 4 → 8 (2x hızlanma)
GRAD_ACCUM = 2             # 32 → 16 (effective=128 aynı)

# 🔧 REGULARİZASYON
WEIGHT_DECAY = 0.05         # ✅ Aynı
WARMUP_RATIO = 0.1          # ✅ Aynı
MAX_GRAD_NORM = 1.0         # ✅ Aynı (0.5 → 1.0)

# 🎛️ LoRA - ⚡ DROPOUT KAPALI (hız için)
LORA_R = 32
LORA_ALPHA = 64
LORA_DROPOUT = 0.05          # 0.1 → 0.0 ⚡ (hız için)

print("\n⚙️ AYARLAR")
print("MODEL:", MODEL_NAME)
print("MAX_SEQ_LENGTH:", MAX_SEQ_LENGTH)
print("EPOCH:", NUM_EPOCHS)
print("LR:", LEARNING_RATE)
print("BATCH:", BATCH_SIZE, "GRAD_ACCUM:", GRAD_ACCUM, "EFFECTIVE:", BATCH_SIZE * GRAD_ACCUM)
print("DROPOUT:", LORA_DROPOUT, "⚡ (hız optimizasyonu)")
print("OUTPUT:", OUTPUT_DIR)


⚙️ AYARLAR
MODEL: unsloth/Meta-Llama-3.1-8B-Instruct
MAX_SEQ_LENGTH: 1536
EPOCH: 3
LR: 2.5e-05
BATCH: 8 GRAD_ACCUM: 2 EFFECTIVE: 16
DROPOUT: 0.05 ⚡ (hız optimizasyonu)
OUTPUT: ./mody-llama-son


In [4]:
from unsloth import FastLanguageModel

# Cache temizle (önceki model kalıntılarını temizle)
torch.cuda.empty_cache()

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=torch.bfloat16,
    load_in_4bit=False,
)
print("✅ Model yüklendi")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

✅ Model yüklendi


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj",
    ],
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,  # ⚡ 0.0 (hız için)
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)
model.print_trainable_parameters()
print("✅ LoRA uygulandı")

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 83,886,080 || all params: 8,114,147,328 || trainable%: 1.0338
✅ LoRA uygulandı


In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
tokenizer.padding_side = "right"  # ⚡ Padding side ayarlandı
print("✅ Chat template ayarlandı")
print("✅ Padding side:", tokenizer.padding_side)

✅ Chat template ayarlandı
✅ Padding side: right


In [7]:
if not os.path.exists(JSONL_FILE):
    raise FileNotFoundError(f"❌ Veri yok: {JSONL_FILE}")

raw_data = []
with open(JSONL_FILE, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            raw_data.append(json.loads(line))

print(f"✅ Toplam diyalog: {len(raw_data):,}")

✅ Toplam diyalog: 5,830


In [8]:
# Dataset: sadece messages
dataset = Dataset.from_list([{"messages": item["messages"]} for item in raw_data])
split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split["train"]
val_dataset = split["test"]

print("Train:", len(train_dataset), "Val:", len(val_dataset))
print("Columns:", train_dataset.column_names)

Train: 4664 Val: 1166
Columns: ['messages']


In [9]:
def formatting_func(examples):
    convos = examples["messages"]
    texts = []
    for convo in convos:
        t = tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False,
        )
        # template preamble temizliği
        t = t.replace("Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n", "")
        t = t.replace("Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n", "")
        texts.append(t)
    return {"text": texts}

train_text = train_dataset.map(formatting_func, batched=True, remove_columns=["messages"])
val_text   = val_dataset.map(formatting_func, batched=True, remove_columns=["messages"])

print("✅ text üretildi")
print("Örnek text (ilk 400 char):\n", train_text[0]["text"][:400])
print("Columns:", train_text.column_names)

Map:   0%|          | 0/4664 [00:00<?, ? examples/s]

Map:   0%|          | 0/1166 [00:00<?, ? examples/s]

✅ text üretildi
Örnek text (ilk 400 char):
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Sen MODY (Maturity Onset Diabetes of the Young) hesaplayici chatbot'sun. Exeter Universitesi'nin gelistirdigi MODY Calculator'in Turkce versiyonusun. Mucahit Yalcinkaya tarafindan gelistirildin.

GOREV: Kullanicidan bilgi toplayarak MODY olasiligini hesaplamak icin gerekli verileri JSON formatinda cikart.

TOPLANACAK BILGILER:
1. Diyabet
Columns: ['text']


In [10]:
def tok_fn(examples):
    out = tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        add_special_tokens=False,
    )
    return out

train_tok = train_text.map(tok_fn, batched=True, remove_columns=["text"])
val_tok   = val_text.map(tok_fn, batched=True, remove_columns=["text"])

print("✅ Tokenize bitti")
print("Columns:", train_tok.column_names)

# Token uzunluk istatistiği
N = min(300, len(train_tok))
lens = [len(train_tok[i]["input_ids"]) for i in range(N)]
print("Min:", min(lens), "Median:", int(np.median(lens)), "P90:", int(np.percentile(lens, 90)), "Max:", max(lens))

Map:   0%|          | 0/4664 [00:00<?, ? examples/s]

Map:   0%|          | 0/1166 [00:00<?, ? examples/s]

✅ Tokenize bitti
Columns: ['input_ids', 'attention_mask']
Min: 982 Median: 1181 P90: 1269 Max: 1357


In [11]:
# ⚡⚡⚡ KRİTİK: RandomEvalSubsetCallback yerine direkt subset
# Ama her evaluate'te FARKLI 200 seçmek için callback kullanacağız

print("="*60)
print("⚡ EVAL DATASET OPTİMİZASYONU")
print("="*60)
print(f"Original eval size: {len(val_tok)}")
print(f"Target subset size: 200 (her eval'de farklı)")
print("="*60)

# Not: Callback trainer oluşturulduktan sonra eklenecek
# Şimdilik val_tok'u olduğu gibi bırakıyoruz

⚡ EVAL DATASET OPTİMİZASYONU
Original eval size: 1166
Target subset size: 200 (her eval'de farklı)


In [12]:
from transformers import DataCollatorForLanguageModeling

class DataCollatorForAssistantOnly_Robust(DataCollatorForLanguageModeling):
    """
    Input:  system + user + assistant (tüm konuşma)
    Loss:   SADECE assistant içerikleri
    """
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer, mlm=False)
        self.tokenizer = tokenizer
        self.start_header_id = tokenizer.convert_tokens_to_ids("<|start_header_id|>")
        self.end_header_id   = tokenizer.convert_tokens_to_ids("<|end_header_id|>")
        if self.start_header_id is None or self.end_header_id is None:
            raise ValueError("Tokenizer <|start_header_id|>/<|end_header_id|> tokenlarını tanımıyor.")
        self.newline_ids = tokenizer.encode("\n", add_special_tokens=False)

    def torch_call(self, examples):
        batch = super().torch_call(examples)
        input_ids = batch["input_ids"]
        labels = batch["labels"].clone()

        for b in range(input_ids.size(0)):
            ids = input_ids[b].tolist()
            labels[b].fill_(-100)

            i = 0
            L = len(ids)
            while i < L:
                if ids[i] != self.start_header_id:
                    i += 1
                    continue

                j = i + 1
                while j < L and ids[j] != self.end_header_id:
                    j += 1
                if j >= L:
                    break

                role_token_ids = ids[i+1:j]
                role_text = self.tokenizer.decode(role_token_ids, skip_special_tokens=False).strip()

                content_start = j + 1
                while content_start < L and ids[content_start] in self.newline_ids:
                    content_start += 1

                content_end = content_start
                while content_end < L and ids[content_end] != self.start_header_id:
                    content_end += 1

                if role_text == "assistant":
                    labels[b, content_start:content_end] = input_ids[b, content_start:content_end]

                i = content_end

        batch["labels"] = labels
        return batch

data_collator = DataCollatorForAssistantOnly_Robust(tokenizer)
print("✅ Collator hazır")

✅ Collator hazır


In [13]:
ex = train_tok[0]
tmp = data_collator([{"input_ids": ex["input_ids"]}])

labels = tmp["labels"][0]
num_supervised = (labels != -100).sum().item()
total = labels.numel()

print("="*60)
print("🧪 MASKING SANITY CHECK (single example)")
print("="*60)
print("Supervised:", num_supervised, "Total:", total, "Ratio:", num_supervised/total)

if num_supervised == 0:
    raise RuntimeError("❌ Single example'da bile supervise yok")
else:
    print("✅ Single example masking OK")

🧪 MASKING SANITY CHECK (single example)
Supervised: 60 Total: 1065 Ratio: 0.056338028169014086
✅ Single example masking OK


In [14]:
from transformers import TrainingArguments, Trainer
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,

    # ⚡ BATCH - 80GB için optimize
    per_device_train_batch_size=BATCH_SIZE,      # 4 → 8
    per_device_eval_batch_size=4,                # 12 → 24 ⚡
    gradient_accumulation_steps=GRAD_ACCUM,      # 32 → 16

    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,

    weight_decay=WEIGHT_DECAY,
    max_grad_norm=MAX_GRAD_NORM,

    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),

    # ⚡ LOGGING - Daha az sık
    logging_steps=25,                             # 10 → 25 ⚡
    logging_first_step=True,

    # ⚡⚡⚡ EVAL - ÇOK DAHA SEYREK
    eval_strategy="steps",
    eval_steps=50,                               # 10 → 100 ⚡⚡⚡

    # ⚡ SAVE - eval_steps ile AYNI olmalı (load_best_model_at_end için)
    save_strategy="steps",
    save_steps=100,                               # 200 → 100 (eval ile aynı) ✅
    save_total_limit=2,                           # 3 → 2
    load_best_model_at_end=True,                  # ✅ Artık çalışır
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    optim="adamw_8bit",

    seed=42,
    report_to="none",
    dataloader_num_workers=8,                     # 4 → 8 ⚡
    prediction_loss_only=True,
    dataloader_pin_memory=True,
)

print("✅ Training arguments hazır")
print("⚡ Batch size: 8 (gradient offload önlendi)")
print("⚡ Eval steps: 100 (10x daha az eval)")
print("⚡ Save steps: 100 (eval ile aynı - best model loading için)")
print("⚡ Eval batch: 24 (2x daha hızlı eval)")
print("⚡ Workers: 8 (2x daha hızlı data loading)")

✅ Training arguments hazır
⚡ Batch size: 8 (gradient offload önlendi)
⚡ Eval steps: 100 (10x daha az eval)
⚡ Save steps: 100 (eval ile aynı - best model loading için)
⚡ Eval batch: 24 (2x daha hızlı eval)
⚡ Workers: 8 (2x daha hızlı data loading)


In [15]:
from transformers import TrainerCallback

# Trainer oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,  # Başlangıçta tam dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("✅ Transformers Trainer hazır")

# 📊 PERPLEXITY CALLBACK
class PerplexityCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            if "loss" in logs:
                train_ppl = np.exp(logs["loss"])
                logs["train_perplexity"] = train_ppl
                print(f"📈 Train Loss: {logs['loss']:.4f} | PPL: {train_ppl:.2f}")

            if "eval_loss" in logs:
                eval_ppl = np.exp(logs["eval_loss"])
                logs["eval_perplexity"] = eval_ppl
                print(f"📊 Eval Loss: {logs['eval_loss']:.4f} | PPL: {eval_ppl:.2f}")

# ⚡⚡⚡ DİNAMİK EVAL SUBSET CALLBACK (Her seferinde farklı 200!)
class DynamicEvalSubsetCallback(TrainerCallback):
    def __init__(self, full_eval_dataset, subset_size=200, seed=42):
        self.full_eval_dataset = full_eval_dataset
        self.subset_size = subset_size
        self.rng = np.random.default_rng(seed)
        self.eval_count = 0

    def on_step_end(self, args, state, control, **kwargs):
        # Eval step'inden hemen ÖNCE subset seç
        if state.global_step > 0 and state.global_step % args.eval_steps == 0:
            n = len(self.full_eval_dataset)
            k = min(self.subset_size, n)

            # HER SEFERINDE FARKLI rastgele indeks seç
            idx = self.rng.choice(n, size=k, replace=False).tolist()

            # Trainer'ın eval_dataset'ini güncelle
            import sys
            for obj in sys.modules['__main__'].__dict__.values():
                if isinstance(obj, Trainer):
                    obj.eval_dataset = self.full_eval_dataset.select(idx)
                    self.eval_count += 1
                    print(f"⚡ Eval #{self.eval_count}: Rastgele 200 örnek seçildi")
                    break

# CALLBACK'LERİ EKLE
trainer.add_callback(PerplexityCallback())
trainer.add_callback(DynamicEvalSubsetCallback(
    full_eval_dataset=val_tok,
    subset_size=200,  # 1000 → 200 ⚡⚡⚡
    seed=42,
))

print("✅ Perplexity tracking eklendi")
print("✅ Dinamik eval subset (her seferinde farklı 200) eklendi")

/tmp/ipython-input-1757044219.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Transformers Trainer hazır
✅ Perplexity tracking eklendi
✅ Dinamik eval subset (her seferinde farklı 200) eklendi


In [16]:
print("="*60)
print("🧪 DATALOADER MASKING CHECK")
print("="*60)

dl = trainer.get_train_dataloader()
batch = next(iter(dl))

print("batch seq_len:", batch["input_ids"].shape[1])

labels = batch["labels"]
num_supervised = (labels != -100).sum().item()
total = labels.numel()

print("Loss'a giren token:", num_supervised)
print("Toplam token:", total)
print("Oran:", num_supervised/total)

if num_supervised == 0:
    raise RuntimeError("❌ Masking hatalı")
else:
    print("✅ Dataloader masking OK")

🧪 DATALOADER MASKING CHECK
batch seq_len: 1241
Loss'a giren token: 1637
Toplam token: 9928
Oran: 0.16488718775181305
✅ Dataloader masking OK


In [17]:
print("="*60)
print("🚀 TRAINING BAŞLIYOR")
print("="*60)
print("⚡ OPTİMİZASYONLAR:")
print("  - Batch size: 4 → 8 (gradient offload yok)")
print("  - Eval steps: 10 → 100 (10x daha az)")
print("  - Eval subset: 1000 → 200 (5x daha hızlı, her seferinde farklı)")
print("  - Dropout: 0.1 → 0.0 (hız için)")
print("  - Workers: 4 → 8 (data loading hızı)")
print("  - Eval batch: 12 → 24 (eval hızı)")
print("="*60)
print("Beklenen süre: ~20-25 dakika (1 epoch)")
print("="*60)

train_out = trainer.train()
print("✅ Train bitti:", train_out)

🚀 TRAINING BAŞLIYOR
⚡ OPTİMİZASYONLAR:
  - Batch size: 4 → 8 (gradient offload yok)
  - Eval steps: 10 → 100 (10x daha az)
  - Eval subset: 1000 → 200 (5x daha hızlı, her seferinde farklı)
  - Dropout: 0.1 → 0.0 (hız için)
  - Workers: 4 → 8 (data loading hızı)
  - Eval batch: 12 → 24 (eval hızı)
Beklenen süre: ~20-25 dakika (1 epoch)


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,664 | Num Epochs = 3 | Total steps = 876
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss,Perplexity
50,10.436100,3.230895,25.302293
100,5.776100,2.570342,13.070295
150,5.561500,2.459773,11.702156
200,5.304600,2.342589,10.408150
250,5.373700,2.570022,13.066110
300,5.313700,2.306090,10.035109
350,4.790600,2.182937,8.872325
400,5.097600,2.388271,10.894642
450,5.733100,2.374235,10.742796
500,5.025600,2.395568,10.974426


📈 Train Loss: 14.6879 | PPL: 2392621.06
📈 Train Loss: 16.8231 | PPL: 20238554.87
⚡ Eval #1: Rastgele 200 örnek seçildi
📈 Train Loss: 10.4361 | PPL: 34067.53


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


📊 Eval Loss: 3.2309 | PPL: 25.30
📈 Train Loss: 6.0467 | PPL: 422.72
⚡ Eval #2: Rastgele 200 örnek seçildi
📈 Train Loss: 5.7761 | PPL: 322.50
📊 Eval Loss: 2.5703 | PPL: 13.07
📈 Train Loss: 5.2592 | PPL: 192.33
⚡ Eval #3: Rastgele 200 örnek seçildi
📈 Train Loss: 5.5615 | PPL: 260.21
📊 Eval Loss: 2.4598 | PPL: 11.70
📈 Train Loss: 5.3254 | PPL: 205.49
⚡ Eval #4: Rastgele 200 örnek seçildi
📈 Train Loss: 5.3046 | PPL: 201.26
📊 Eval Loss: 2.3426 | PPL: 10.41
📈 Train Loss: 5.4720 | PPL: 237.94
⚡ Eval #5: Rastgele 200 örnek seçildi
📈 Train Loss: 5.3737 | PPL: 215.66
📊 Eval Loss: 2.5700 | PPL: 13.07
📈 Train Loss: 4.9507 | PPL: 141.27
⚡ Eval #6: Rastgele 200 örnek seçildi
📈 Train Loss: 5.3137 | PPL: 203.10
📊 Eval Loss: 2.3061 | PPL: 10.04
📈 Train Loss: 5.6288 | PPL: 278.33
⚡ Eval #7: Rastgele 200 örnek seçildi
📈 Train Loss: 4.7906 | PPL: 120.37
📊 Eval Loss: 2.1829 | PPL: 8.87
📈 Train Loss: 5.7850 | PPL: 325.38
⚡ Eval #8: Rastgele 200 örnek seçildi
📈 Train Loss: 5.0976 | PPL: 163.63
📊 Eval Loss: 2

In [20]:

# Eğitim bittikten sonra çalıştır
# ════════════════════════════════════════════════════════════════

import os
import shutil
import torch
from google.colab import drive
from unsloth import FastLanguageModel
from peft import PeftModel

# Drive mount
drive.mount("/content/drive")

# ════════════════════════════════════════════════════════════════
# AYARLAR
# ════════════════════════════════════════════════════════════════

CKPT_DIR = "/content/mody-llama-son"
MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct"
MAX_SEQ_LENGTH = 1536

MERGED_DIR = "/content/mody"
DRIVE_OUT = "/content/drive/MyDrive/mody_son"

os.makedirs(MERGED_DIR, exist_ok=True)
os.makedirs(DRIVE_OUT, exist_ok=True)

# ════════════════════════════════════════════════════════════════
# 1. En son checkpoint'u bul
# ════════════════════════════════════════════════════════════════

checkpoints = [d for d in os.listdir(CKPT_DIR) if d.startswith("checkpoint-")]
if not checkpoints:
    raise RuntimeError("❌ Checkpoint bulunamadı!")

latest_ckpt = max(checkpoints, key=lambda x: int(x.split("-")[1]))
CKPT_PATH = os.path.join(CKPT_DIR, latest_ckpt)
print(f"✅ Checkpoint: {CKPT_PATH}")

# ════════════════════════════════════════════════════════════════
# 2. Base model + LoRA yükle
# ════════════════════════════════════════════════════════════════

torch.cuda.empty_cache()

print("🔄 Base model yükleniyor...")
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=torch.bfloat16,
    load_in_4bit=False,
)

print("🔄 LoRA adapter yükleniyor...")
model = PeftModel.from_pretrained(base_model, CKPT_PATH)
model.eval()

# ════════════════════════════════════════════════════════════════
# 3. Merge
# ════════════════════════════════════════════════════════════════

print("🔄 Merge ediliyor...")
merged_model = model.merge_and_unload()

print(f"💾 Kaydediliyor: {MERGED_DIR}")
merged_model.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print("✅ Merge tamamlandı!")

# ════════════════════════════════════════════════════════════════
# 4. ZIP oluştur
# ════════════════════════════════════════════════════════════════

print("📦 ZIP oluşturuluyor...")
zip_path = shutil.make_archive("/content/mody", "zip", MERGED_DIR)
print(f"✅ ZIP: {zip_path}")

# ════════════════════════════════════════════════════════════════
# 5. Drive'a kopyala
# ════════════════════════════════════════════════════════════════

drive_zip = os.path.join(DRIVE_OUT, "mody.zip")
shutil.copy2(zip_path, drive_zip)

print("="*60)
print("🎉 TAMAMLANDI!")
print("="*60)
print(f"📁 Merged: {MERGED_DIR}")
print(f"📦 ZIP: {zip_path}")
print(f"🚀 Drive: {drive_zip}")
print("="*60)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Checkpoint: /content/mody-llama-son/checkpoint-876
🔄 Base model yükleniyor...
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

🔄 LoRA adapter yükleniyor...
🔄 Merge ediliyor...
💾 Kaydediliyor: /content/mody
✅ Merge tamamlandı!
📦 ZIP oluşturuluyor...
✅ ZIP: /content/mody.zip
🎉 TAMAMLANDI!
📁 Merged: /content/mody
📦 ZIP: /content/mody.zip
🚀 Drive: /content/drive/MyDrive/mody_son/mody.zip
